# TP3 Astrometría - Configuración de Base de Datos SQL

Este notebook demuestra cómo configurar y ejecutar SQL en Jupyter con Python.

## 1. Instalar Paquetes SQL Requeridos

Para ejecutar SQL en tu kernel de Python, tienes **dos opciones principales**:

In [1]:
# OPCIÓN 1: Usar SQLite3 (YA INCLUIDO en Python, no necesitas instalar nada)
import sqlite3
print("SQLite3 ya está disponible en Python por defecto")
print(f"Versión SQLite: {sqlite3.sqlite_version}")

SQLite3 ya está disponible en Python por defecto
Versión SQLite: 3.50.2


In [2]:
# OPCIÓN 2: Instalar ipython-sql para comandos mágicos %%sql (OPCIONAL)
# Descomenta las siguientes líneas si quieres usar magic commands:

# !pip install ipython-sql sqlalchemy
print("Para usar comandos mágicos %%sql, ejecuta: pip install ipython-sql sqlalchemy")

Para usar comandos mágicos %%sql, ejecuta: pip install ipython-sql sqlalchemy


## 2. Configurar Conexión a Base de Datos

In [3]:
import sqlite3
import os

# Crear conexión a base de datos SQLite
db_path = 'tp3_astrometria.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"✓ Conexión establecida con la base de datos: {db_path}")
print(f"✓ Base de datos ubicada en: {os.path.abspath(db_path)}")

✓ Conexión establecida con la base de datos: tp3_astrometria.db
✓ Base de datos ubicada en: /home/rubsanmon/Escritorio/facultad/4to/2c/astrometria1/2025-unc-famaf-astro-astrometria1/tp3/tp3_astrometria.db


## 3. Cargar Extensión SQL Magic (Opcional)

Si instalaste `ipython-sql`, puedes usar comandos mágicos:

In [4]:
# Si instalaste ipython-sql, descomenta estas líneas:
# %load_ext sql
# %sql sqlite:///tp3_astrometria.db

print("Magic commands disponibles después de instalar ipython-sql:")
print("- %sql para líneas individuales")
print("- %%sql para celdas completas")

Magic commands disponibles después de instalar ipython-sql:
- %sql para líneas individuales
- %%sql para celdas completas


## 4. Crear Tablas de Base de Datos

In [5]:
# Crear tabla Estrellas
cursor.execute('''
CREATE TABLE IF NOT EXISTS Estrellas (
    id_estrella INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    tipo_espectral TEXT,
    distancia_pc REAL,
    magnitud REAL
)
''')

print("✓ Tabla 'Estrellas' creada exitosamente")

✓ Tabla 'Estrellas' creada exitosamente


In [6]:
# Crear tabla Planetas
cursor.execute('''
CREATE TABLE IF NOT EXISTS Planetas (
    id_planeta INTEGER PRIMARY KEY,
    id_estrella INTEGER,
    nombre TEXT NOT NULL,
    masa REAL,
    periodo_orbital REAL,
    descubrimiento TEXT,
    FOREIGN KEY (id_estrella) REFERENCES Estrellas(id_estrella)
)
''')

conn.commit()
print("✓ Tabla 'Planetas' creada exitosamente")
print("✓ Cambios guardados en la base de datos")

✓ Tabla 'Planetas' creada exitosamente
✓ Cambios guardados en la base de datos


## 5. Insertar Datos de Muestra

In [7]:
# Insertar estrellas de ejemplo
estrellas_ejemplo = [
    (1, 'Sol', 'G2V', 0.000004848, -26.7),
    (2, 'Proxima Centauri', 'M5.5Ve', 1.3, 11.13),
    (3, 'Sirius A', 'A1V', 2.64, -1.46),
    (4, 'Kepler-452', 'G2V', 430, 13.4)
]

cursor.executemany('''
INSERT OR REPLACE INTO Estrellas 
(id_estrella, nombre, tipo_espectral, distancia_pc, magnitud)
VALUES (?, ?, ?, ?, ?)
''', estrellas_ejemplo)

print(f"✓ {len(estrellas_ejemplo)} estrellas insertadas")

✓ 4 estrellas insertadas


In [8]:
# Insertar planetas de ejemplo
planetas_ejemplo = [
    (1, 1, 'Tierra', 1.0, 365.25, '1543 (Copérnico)'),
    (2, 1, 'Marte', 0.107, 687, '1543 (Copérnico)'),
    (3, 2, 'Proxima Centauri b', 1.17, 11.18, '2016'),
    (4, 4, 'Kepler-452b', 5.0, 384.8, '2015')
]

cursor.executemany('''
INSERT OR REPLACE INTO Planetas 
(id_planeta, id_estrella, nombre, masa, periodo_orbital, descubrimiento)
VALUES (?, ?, ?, ?, ?, ?)
''', planetas_ejemplo)

conn.commit()
print(f"✓ {len(planetas_ejemplo)} planetas insertados")

✓ 4 planetas insertados


## 6. Ejecutar Consultas SQL

In [9]:
# Consulta 1: Listar todas las estrellas
print("=== TODAS LAS ESTRELLAS ===")
cursor.execute('SELECT * FROM Estrellas')
estrellas = cursor.fetchall()

for estrella in estrellas:
    print(f"ID: {estrella[0]}, Nombre: {estrella[1]}, Tipo: {estrella[2]}, Distancia: {estrella[3]} pc, Magnitud: {estrella[4]}")

=== TODAS LAS ESTRELLAS ===
ID: 1, Nombre: Sol, Tipo: G2V, Distancia: 4.848e-06 pc, Magnitud: -26.7
ID: 2, Nombre: Proxima Centauri, Tipo: M5.5Ve, Distancia: 1.3 pc, Magnitud: 11.13
ID: 3, Nombre: Sirius A, Tipo: A1V, Distancia: 2.64 pc, Magnitud: -1.46
ID: 4, Nombre: Kepler-452, Tipo: G2V, Distancia: 430.0 pc, Magnitud: 13.4


In [10]:
# Consulta 2: Listar planetas con información de su estrella
print("\n=== PLANETAS CON SUS ESTRELLAS ===")
cursor.execute('''
SELECT p.nombre as planeta, e.nombre as estrella, p.masa, p.periodo_orbital, p.descubrimiento
FROM Planetas p
JOIN Estrellas e ON p.id_estrella = e.id_estrella
''')

planetas = cursor.fetchall()
for planeta in planetas:
    print(f"Planeta: {planeta[0]}, Estrella: {planeta[1]}, Masa: {planeta[2]} M⊕, Período: {planeta[3]} días, Descubrimiento: {planeta[4]}")


=== PLANETAS CON SUS ESTRELLAS ===
Planeta: Tierra, Estrella: Sol, Masa: 1.0 M⊕, Período: 365.25 días, Descubrimiento: 1543 (Copérnico)
Planeta: Marte, Estrella: Sol, Masa: 0.107 M⊕, Período: 687.0 días, Descubrimiento: 1543 (Copérnico)
Planeta: Proxima Centauri b, Estrella: Proxima Centauri, Masa: 1.17 M⊕, Período: 11.18 días, Descubrimiento: 2016
Planeta: Kepler-452b, Estrella: Kepler-452, Masa: 5.0 M⊕, Período: 384.8 días, Descubrimiento: 2015


In [11]:
# Consulta 3: Estrellas más cercanas (menos de 10 parsecs)
print("\n=== ESTRELLAS CERCANAS (<10 pc) ===")
cursor.execute('''
SELECT nombre, distancia_pc, magnitud 
FROM Estrellas 
WHERE distancia_pc < 10 
ORDER BY distancia_pc
''')

cercanas = cursor.fetchall()
for estrella in cercanas:
    print(f"{estrella[0]}: {estrella[1]} pc, magnitud {estrella[2]}")


=== ESTRELLAS CERCANAS (<10 pc) ===
Sol: 4.848e-06 pc, magnitud -26.7
Proxima Centauri: 1.3 pc, magnitud 11.13
Sirius A: 2.64 pc, magnitud -1.46


## 7. Verificar Configuración de Base de Datos

In [12]:
# Verificar estructura de las tablas
print("=== ESTRUCTURA DE LA BASE DE DATOS ===")

# Listar todas las tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()
print(f"Tablas en la base de datos: {[tabla[0] for tabla in tablas]}")

# Verificar estructura de cada tabla
for tabla in tablas:
    nombre_tabla = tabla[0]
    print(f"\n--- Estructura de {nombre_tabla} ---")
    cursor.execute(f"PRAGMA table_info({nombre_tabla})")
    columnas = cursor.fetchall()
    for col in columnas:
        print(f"  {col[1]} ({col[2]}) - PK: {bool(col[5])} - NotNull: {bool(col[3])}")

=== ESTRUCTURA DE LA BASE DE DATOS ===
Tablas en la base de datos: ['Estrellas', 'Planetas']

--- Estructura de Estrellas ---
  id_estrella (INTEGER) - PK: True - NotNull: False
  nombre (TEXT) - PK: False - NotNull: True
  tipo_espectral (TEXT) - PK: False - NotNull: False
  distancia_pc (REAL) - PK: False - NotNull: False
  magnitud (REAL) - PK: False - NotNull: False

--- Estructura de Planetas ---
  id_planeta (INTEGER) - PK: True - NotNull: False
  id_estrella (INTEGER) - PK: False - NotNull: False
  nombre (TEXT) - PK: False - NotNull: True
  masa (REAL) - PK: False - NotNull: False
  periodo_orbital (REAL) - PK: False - NotNull: False
  descubrimiento (TEXT) - PK: False - NotNull: False


In [13]:
# Contar registros en cada tabla
print("\n=== CONTEO DE REGISTROS ===")

cursor.execute('SELECT COUNT(*) FROM Estrellas')
count_estrellas = cursor.fetchone()[0]
print(f"Estrellas: {count_estrellas} registros")

cursor.execute('SELECT COUNT(*) FROM Planetas')
count_planetas = cursor.fetchone()[0]
print(f"Planetas: {count_planetas} registros")

print("\n✓ Base de datos configurada y funcionando correctamente")


=== CONTEO DE REGISTROS ===
Estrellas: 4 registros
Planetas: 4 registros

✓ Base de datos configurada y funcionando correctamente


In [14]:
# Cerrar la conexión
conn.close()
print("✓ Conexión a base de datos cerrada")

print("\n🎉 ¡Configuración completada!")
print("\nTu base de datos está lista. Puedes usar:")
print("1. sqlite3 (incluido en Python) - para SQL básico")
print("2. ipython-sql (opcional) - para comandos mágicos %%sql")
print("3. pandas + sqlite3 - para análisis de datos")

✓ Conexión a base de datos cerrada

🎉 ¡Configuración completada!

Tu base de datos está lista. Puedes usar:
1. sqlite3 (incluido en Python) - para SQL básico
2. ipython-sql (opcional) - para comandos mágicos %%sql
3. pandas + sqlite3 - para análisis de datos
